In [44]:
import altair as alt
import pandas as pd
import geopandas as gpd # Requires geopandas -- e.g.: conda install -c conda-forge geopandas
import toolz

import os

def custom(data):
    return toolz.curried.pipe(data, alt.to_json(filename='altdata/{prefix}-{hash}.{extension}') )
alt.data_transformers.register('custom', custom)
alt.data_transformers.enable('custom')

pass

In [45]:
df = pd.read_csv("dpt2020.csv", sep=";")
df.drop(df[df.preusuel == '_PRENOMS_RARES'].index, inplace=True)
df.drop(df[df.dpt == 'XX'].index, inplace=True)
df['annais'] = df['annais'].astype(int)
df['nombre'] = df['nombre'].astype(int)
df['sexe'] = df['sexe'].replace({1: 'M', 2: 'F'})
df = df.rename(columns={"annais": "annee"})

In [47]:
periode_debut = 2010
periode_fin = 2015
df_filtered = df[(df['annee'] >= periode_debut) & (df['annee'] <= periode_fin)]

df_total = df_filtered.groupby(['preusuel', 'sexe'], as_index=False)['nombre'].sum()

# on considère le top 250, pour éviter les prenoms donnés une seule fois par exemple
top_250_boys = df_total[df_total['sexe'] == 'M'].nlargest(250, 'nombre')
top_250_girls = df_total[df_total['sexe'] == 'F'].nlargest(250, 'nombre')
filtered_names = pd.concat([top_250_boys, top_250_girls])['preusuel'].unique()
df_filtered = df_filtered[df_filtered['preusuel'].isin(filtered_names)]

df_grouped = df_filtered.groupby(['preusuel', 'sexe', 'annee'], as_index=False)['nombre'].sum()

# nombre moyen d'occurence sur la période
df_mean = df_grouped.groupby(['preusuel', 'sexe'], as_index=False)['nombre'].mean().rename(columns={'nombre': 'nombre_moyen'})

# variation annuelle pour chaque prénom
df_grouped['variation'] = df_grouped.groupby(['preusuel', 'sexe'])['nombre'].diff()
df_grouped['variation_absolue'] = df_grouped['variation'].abs()


df_grouped = pd.merge(df_grouped, df_mean, on=['preusuel', 'sexe'])   # Fusionner avec le nombre moyen d'occurrences

# on normalie la variation absolue par le nombre moyen d'occurrences
df_grouped['variation_normalisee'] = df_grouped['variation_absolue'] / df_grouped['nombre_moyen'] 

df_aggregated = df_grouped.groupby(['preusuel', 'sexe'], as_index=False)['variation_normalisee'].mean()

top_10_boys = df_aggregated[df_aggregated['sexe'] == 'M'].nsmallest(10, 'variation_normalisee')
top_10_girls = df_aggregated[df_aggregated['sexe'] == 'F'].nsmallest(10, 'variation_normalisee')

top_10_boys = top_10_boys.sort_values(by='variation_normalisee', ascending=False)
top_10_girls = top_10_girls.sort_values(by='variation_normalisee', ascending=False)


alternated_list = []
for i in range(max(len(top_10_boys), len(top_10_girls))):
    if i < len(top_10_boys):
        alternated_list.append(top_10_boys.iloc[i])
    if i < len(top_10_girls):
        alternated_list.append(top_10_girls.iloc[i])



df_alternated = pd.DataFrame(alternated_list)


chart = alt.Chart(df_alternated).mark_bar().encode(
    x=alt.X('preusuel:N', title='Prénom', sort=None),
    y=alt.Y('variation_normalisee:Q', title='Variation Normalisée'),
    color=alt.Color('sexe:N', scale=alt.Scale(domain=['M', 'F'], range=['blue', 'pink']), title='Sexe'),
    xOffset=alt.XOffset("sexe:N")
).properties(
    width=800
)

chart


alt.Chart(...)